<a href="https://colab.research.google.com/github/janhavidusad21/Data-science-Assigment/blob/main/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
anime=pd.read_csv("/content/anime.csv")

In [3]:
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [4]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


Handling Missing Values

In [5]:
anime.isnull().sum()

,0
anime_id,0
name,0
genre,62
type,25
episodes,0
rating,230
members,0


In [6]:
anime["genre"]=anime["genre"].fillna(anime["genre"].mode()[0])

In [7]:
anime["type"]=anime["type"].fillna(anime["type"].mode()[0])

In [8]:
anime["rating"]=anime["rating"].fillna(anime["rating"].mean())

In [9]:
anime.isnull().sum()

,0
anime_id,0
name,0
genre,0
type,0
episodes,0
rating,0
members,0


In [24]:
# Create genre dummy variables
anime['genre'] = anime['genre'].fillna('')  # replace NaN with empty string
genres = anime['genre'].str.get_dummies(sep=', ')

# Combine with main dataframe
anime_features = pd.concat([anime[['name', 'rating']], genres], axis=1)


In [27]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
anime_features['rating'] = scaler.fit_transform(anime_features[['rating']])


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

# Drop the 'name' column for similarity computation
features_matrix = anime_features.drop('name', axis=1)

# Compute similarity matrix
cosine_sim = cosine_similarity(features_matrix)


In [29]:
import numpy as np

def recommend_anime(anime_name, df, similarity_matrix, top_n=5):
    # Find index of target anime
    idx = df[df['name'] == anime_name].index[0]

    # Get similarity scores
    sim_scores = list(enumerate(similarity_matrix[idx]))

    # Sort by similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get indices of top matches excluding itself
    sim_scores = sim_scores[1:top_n+1]

    anime_indices = [i[0] for i in sim_scores]

    return df['name'].iloc[anime_indices]

# Example usage:
print(recommend_anime('Naruto', anime_features, cosine_sim, top_n=5))


615                                    Naruto: Shippuuden
1103    Boruto: Naruto the Movie - Naruto ga Hokage ni...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
Name: name, dtype: object


In [30]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(anime_features, test_size=0.2, random_state=42)


In [31]:
train

,name,rating,'Adventure','Adventure'],'Cars','Cars'],'Comedy','Comedy'],'Dementia','Dementia'],...,['Shounen'],['Slice of Life',['Slice of Life'],['Space'],['Sports'],['Super Power',['Supernatural'],['Thriller'],['Vampire'],['Yaoi']
3013,Asura Cryin&#039;,0.651861,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4253,MM! Specials,0.612245,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9791,Nyani ga Nyandaa Nyandaa Kamen,0.609844,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2629,Touch: Cross Road - Kaze no Yukue,0.665066,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4608,Kurenai Sanshirou,0.601441,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,Milkyway,0.498199,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5191,Tondemo Nezumi Daikatsuyaku,0.583433,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,Macross II: Lovers Again,0.576230,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
860,Aikatsu! Movie,0.734694,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
test

,name,rating,'Adventure','Adventure'],'Cars','Cars'],'Comedy','Comedy'],'Dementia','Dementia'],...,['Shounen'],['Slice of Life',['Slice of Life'],['Space'],['Sports'],['Super Power',['Supernatural'],['Thriller'],['Vampire'],['Yaoi']
6329,Suzy&#039;s Zoo: Daisuki! Witzy - Happy Birthday,0.540216,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2167,Tactics,0.680672,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2882,Kamen no Maid Guy,0.656663,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4700,Take Your Way,0.599040,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7258,Rinkaku,0.471789,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9652,Naita Aka Oni (OVA),0.489796,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
549,Kuroko no Basket 3rd Season NG-shuu,0.757503,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7550,New York Trip,0.436975,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5210,Itsuka Tenma no Kuro Usagi Special,0.582233,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
def precision_recall_at_k(recommended_items, relevant_items, k):
    recommended_at_k = recommended_items[:k]
    relevant_set = set(relevant_items)
    recommended_set = set(recommended_at_k)
    true_positives = len(recommended_set & relevant_set)
    precision = true_positives / k if k > 0 else 0
    recall = true_positives / len(relevant_set) if len(relevant_set) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

# Example:
recommended = ['Naruto', 'Bleach', 'One Piece', 'Death Note', 'Attack on Titan']
relevant = ['One Piece', 'Attack on Titan', 'Fullmetal Alchemist']

precision, recall, f1 = precision_recall_at_k(recommended, relevant, k=5)
print(f'Precision@5: {precision:.2f}')
print(f'Recall@5: {recall:.2f}')
print(f'F1@5: {f1:.2f}')


Precision@5: 0.40
Recall@5: 0.67
F1@5: 0.50


Low Precision?

Too many irrelevant recommendations → consider adding more features e.g., tags, descriptions

Use hybrid approaches: blend content-based with collaborative filtering.

Low Recall?

Recommender is too narrow → experiment with lowering similarity thresholds to broaden the pool.

Improvements:

Add more discriminative features: studio, release year, popularity.
Use NLP: build a TF-IDF vector on plot descriptions.
Switch to collaborative filtering if you have user behavior data.
Apply dimensionality reduction (SVD, PCA) to improve similarity results.